In [1]:
import pandas as pd
from rapidfuzz import process, fuzz

# Load data from CSV files into DataFrames
df_seasons_statistics_table = pd.read_csv('seasons_statistics_table.csv')
df_seasons_statistics_transfers = pd.read_csv('seasons_statistics_transfers.csv')
df_seasons_statistics_transfers

,Teams,Spend,Balance,Year
0,Chelsea FC,€464.10m,€-186.60m,23/24
1,Tottenham Hotspur,€272.10m,€-151.40m,23/24
2,Manchester City,€259.60m,€-133.80m,23/24
3,Arsenal FC,€235.10m,€-165.90m,23/24
4,Manchester United,€202.30m,€-143.96m,23/24
...,...,...,...,...
395,Blackburn Rovers,€7.88m,€-175k,04/05
396,Middlesbrough FC,€6.75m,€-3.30m,04/05
397,Crystal Palace,€5.15m,€-5.05m,04/05
398,Manchester City,€1.50m,€6.44m,04/05


In [2]:
# Function to clean the specified column in the DataFrame by removing unwanted characters and whitespace
def clean_column_text(df, column_name):
    return (
        df[column_name]
        .str.replace('\n', '', regex=False)
        .str.replace('\xa0', '', regex=False)
        .str.strip()
    )

# Function to match teams between two lists using fuzzy matching
def match_teams(teams_list_1, teams_list_2, threshold=50):
    mapping_dict = {}
    for team in teams_list_1:
        match = process.extractOne(team, teams_list_2, scorer=fuzz.token_sort_ratio)
        if match[1] > threshold:
            mapping_dict[team] = match[0]
        else:
            mapping_dict[team] = None
    return mapping_dict

# Function to convert spend and balance values from strings to numeric values
def convert_currency(value):
    value = str(value).replace('€', '')  # Ensure value is a string and remove euro symbol
    if value in ['-', '', '+-0']:
        return 0.0  # Handle special cases
    elif 'm' in value:
        return float(value.replace('m', '')) * 1e6  # Convert millions
    elif 'k' in value:
        return float(value.replace('k', '')) * 1e3  # Convert thousands
    else:
        return float(value)

def manual_team_adjustments(mapping_dict):
    # Known manual corrections
    corrections = {
        'Wolverhampton Wanderers': 'Wolves',
        'Queens Park Rangers': 'QPR'
    }

    # Apply the corrections
    for incorrect_name, correct_name in corrections.items():
        if incorrect_name in mapping_dict:
            mapping_dict[incorrect_name] = correct_name
        else:
            print(f"Error: '{incorrect_name}' not found in the mapping dictionary.")

    return mapping_dict    
    
    
# Function to process the dataframes
def process_dataframes(df_stats, df_transfers):
    # Clean the 'Teams' column in both dataframes
    df_stats['Teams'] = clean_column_text(df_stats, 'Teams')
    df_transfers['Teams'] = clean_column_text(df_transfers, 'Teams')

    # Extract unique team names from both dataframes
    unique_teams_stats = df_stats['Teams'].unique()
    unique_teams_transfers = df_transfers['Teams'].unique()

    # Perform team name matching between transfers and statistics tables
    mapping_dict = match_teams(unique_teams_transfers, unique_teams_stats)
    
    # Display the mapping dictionary for confirmation
    print("Here is the team mapping dictionary:")
    for team, mapped_team in mapping_dict.items():
        print(f"{team}: {mapped_team}")

    # Ask for user confirmation to continue
    while True:
        user_input = input("Do the team names match correctly? Type 'yes' to continue, 'no' to apply manual corrections, or 'end' to exit: ").lower()
        
        if user_input == 'yes':
            break  # Continue with the function
        elif user_input == 'no':
            # Apply manual corrections and check again
            mapping_dict = manual_team_adjustments(mapping_dict)
            print("Manual corrections have been applied. Here is the updated mapping dictionary:")
            for team, mapped_team in mapping_dict.items():
                print(f"{team}: {mapped_team}")
        elif user_input == 'end':
            print("Exiting the function.")
            return None  # Exit the function
        else:
            print("Input not recognized. Please type 'yes', 'no', or 'end'.")

    # Map the matched team names back to the 'Teams' column in the transfers dataframe
    df_transfers['Teams'] = df_transfers['Teams'].map(mapping_dict).fillna(df_transfers['Teams'])

    # Merge DataFrames on 'Teams' and 'Year' columns to align data and include only common entries
    df_merged = pd.merge(df_stats, df_transfers, on=['Teams', 'Year'], how='inner')

    # Apply the conversion to both Spend and Balance columns in the merged dataframe
    df_merged['Spend'] = df_merged['Spend'].apply(convert_currency)
    df_merged['Balance'] = df_merged['Balance'].apply(convert_currency)

    return df_merged

In [3]:
# Process and merge the dataframes, then store the result in df_merged_data
df_merged_data = process_dataframes(df_seasons_statistics_table, df_seasons_statistics_transfers)
df_merged_data

Here is the team mapping dictionary:
Chelsea FC: Chelsea
Tottenham Hotspur: Tottenham
Manchester City: Man City
Arsenal FC: Arsenal
Manchester United: Man Utd
Liverpool FC: Liverpool
Newcastle United: Newcastle
West Ham United: West Ham
Nottingham Forest: Nottm Forest
AFC Bournemouth: Bournemouth
Aston Villa: Aston Villa
Burnley FC: Burnley
Brighton & Hove Albion: Brighton
Wolverhampton Wanderers: None
Fulham FC: Fulham
Brentford FC: Brentford
Crystal Palace: Crystal Palace
Sheffield United: Sheff Utd
Everton FC: Everton
Luton Town: Luton
Leeds United: Leeds
Southampton FC: Southampton
Leicester City: Leicester
Norwich City: Norwich
Watford FC: Watford
West Bromwich Albion: West Brom
Cardiff City: Cardiff
Huddersfield Town: Huddersfield
Swansea City: Swansea
Stoke City: Stoke City
Middlesbrough FC: Middlesbrough
Sunderland AFC: Sunderland
Hull City: Hull City
Queens Park Rangers: None
Reading FC: Reading
Wigan Athletic: Wigan
Blackburn Rovers: Blackburn
Bolton Wanderers: Bolton
Birming

,Teams,Places,Matches,Wins,Draws,Loses,Goals,Goals Diff,Points,Year,Spend,Balance
0,Man City,1,38,28,7,3,96:34,62,91,23/24,259600000.0,-133800000.0
1,Arsenal,2,38,28,5,5,91:29,62,89,23/24,235100000.0,-165900000.0
2,Liverpool,3,38,24,10,4,86:41,45,82,23/24,172000000.0,-111300000.0
3,Aston Villa,4,38,20,8,10,76:61,15,68,23/24,111500000.0,-78950000.0
4,Tottenham,5,38,20,6,12,74:61,13,66,23/24,272100000.0,-151400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
395,Portsmouth,16,38,10,9,19,43:59,-16,39,04/05,10850000.0,-4250000.0
396,West Brom,17,38,6,16,16,36:61,-25,34,04/05,15850000.0,-13900000.0
397,Crystal Palace,18,38,7,12,19,41:62,-21,33,04/05,5150000.0,-5050000.0
398,Norwich,19,38,7,12,19,42:77,-35,33,04/05,8530000.0,-8080000.0


In [4]:
# Save the merged DataFrame to a CSV file
df_merged_data.to_csv('merged_data.csv', index=False)